<b> `GUID:`</b> 2944377T  
<b> `GitHub Repository:`https://github.com/linetanck/AI_for_the_Arts

# Generating text with Neural Networks

Welcome to this tutorial about generating text with the use of AI. Whether or not this is your first touch with AI or you are quite well read in the topic, you are at the right place. In the following, you will build, train and use an AI model to generate text in the style of William Shakespeare.  
The AI method used are `neural networks`. Neural networks are a type of machine learning process that simmulate the human brain by using connected `nodes` and `layers` instead of neurons.  
Firstly, we will start with preparing the data, then we will build and train the neural network. Lastly, the model can be used by giving the beginning of a Shakespeare quote and the AI will finish it.  
  
<b>Your question might be, why are we doing this?</b>  
Of course it is fun to have a machine generate Shakespeare quotes, if you are well versed in literature it might even be funny to criticise the AI's writing. However, using text and AI has deeper implications. Inspecting the output of a text generation tells us a ot about our own language. Neural networks look at the language data we give it to train in a very different way than we do. A machine does not share all the underlying knowledge about the meaning of each words and their use together. For the neural network the data is abstract and the decisions it makes for the output are based on patters that were observed in the language data. These patterns are super interesting as they reveal things about language that we might not catch ourselves.<br>  <i>So, lets start discovering the capabilities of AI... </i>  
![Shakespeare illustration, saying Lets go](images/Lets_go.png)  
`Image source: own illustration`

![Heading that says "Data"](images/data_heading.png)
`Image source: own illustration`

# 1. Getting the Data

When working with AI, the data is very important, since it has an immense influence on the model. It is its substance. Bad data can mean that the model just does not perform well. Moreover, data that carries `bias` means the model carries bias. As an example, not recognizing racism in data leads to racism in an AI model's output, like in multiple AI controversies over the last few years.  
Although a critical reflection of the data is crucial, it is not neccessary for this notebook and has been done beforehand. The need remains to inspect the data on a technical level though.  
<b>`Code explanation:`</b>   
In the following code, the `librairy` tensorflow is imported. Librairies are code written by other people that you can use for yourself. Tensorflow specifically is necessary to get the dataset and it provides the capabilities to build and train the model.  
After importing tensorflow, the code gets the `dataset` from the shortcut URL and reads it.  
The dataset consists of a Shakespeare text. Maybe copa and paste the link from the code and take a look, its always good to know your data. 

In [ ]:
import tensorflow as tf

shakespeare_url = "https://homl.info/shakespeare"  # shortcut URL
filepath = tf.keras.utils.get_file("shakespeare.txt", shakespeare_url) # getting the file from the URL
with open(filepath) as f:
    shakespeare_text = f.read() # reading the file

It is always important to inspect the data because we need to know what it looks like after reading it in. Executing the following code `prints` an example snippet of how the data looks like. 

In [ ]:
print(shakespeare_text[:80]) # not relevant to machine learning but relevant to exploring the data

Checking the data can already prevent errors happening. It also shows us if all the data was imported correctly. The shape of the data also influences the way it needs to be prepared and fed into the model, which we will be doing in the next step.

# 2. Preparing the Data
There is quite a lot of transforming the data when working with AI.  
For this project, the data is split into characters and then all letters are made lowercase. This is neccessary to later `encode` the data. Encoding is a crucial step when the AI is supposed to work with text. Since the computer handles the data as `binary data` (a collection of 0s and 1s), the text needs to be turned into binary data. This is happening during encoding.  
<b>`Code explanation:`</b>  
The tensorflow library is used to split the dataset into characters and placing everything in lowercase. Then, the data is being encoded.

### Encoding the Data

In [ ]:
text_vec_layer = tf.keras.layers.TextVectorization(split="character",
                                                   standardize="lower")
text_vec_layer.adapt([shakespeare_text])
encoded = text_vec_layer([shakespeare_text])[0]

In the next step, the result of the encoding is printed out again to inspect it. Altough we might not be able to pull much information from the heap of numbers anymore, a few things are shown. The attribute `dtype` shows the data type of the encoded data. Since we need it to be numbers, we have to check if the data type is integers. Moreover, the attribute `shape` shows the size of the data. The second number after the comma is the amount of entries the data has, which will be important when `splitting` the data.

In [ ]:
print(text_vec_layer([shakespeare_text]))

### Tokenizing the Data
The following code transforms the data by droping unwanted values and thereby preparing it for `tokenizing`. This means turning the data into units that are then processed by the AI.

In [ ]:
encoded -= 2  # drop tokens 0 (pad) and 1 (unknown), which we will not use
n_tokens = text_vec_layer.vocabulary_size() - 2  # number of distinct chars = 39
dataset_size = len(encoded)  # total number of chars = 1,115,394

Again, we inspect the result of transforming the data.

In [ ]:
print(n_tokens, dataset_size)

### Further Preparation
In the following, the encoded and tokenized data is prepared further for the AI model. This is very important as the model will not run when the data does not fit what it expects.  
Most importantly, the `batch size` is defined here. The batch size is the amount of entries of the data size the AI model takes one at a time during training. It cant take all of the data at once, so it needs to take a smaller portion.

In [ ]:
def to_dataset(sequence, length, shuffle=False, seed=None, batch_size=32): # batch size is set to 32
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda window_ds: window_ds.batch(length + 1))
    if shuffle:
        ds = ds.shuffle(100_000, seed=seed)
    ds = ds.batch(batch_size)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

### Splitting the Data
The last step necessary for the data is splitting it. There are three processes that the model will run through: `training`, `validation`, and `testing`. For each process we need different data. The concept underlying is to check whether the model is only learning the training data off by heart or if it is actually able to do the same things on unseen new data. In one case, we could just be evaluating the model's knowledge about the data rather than the model's capabilities to generate text. Therefore, we split the data.  
In the following code, the amount of data can be reduced to make the training time shorter. If your computing power is not that much (i.e.: working on a laptop) and you want to make training the model faster, the code indicates which numbers you have to change. <b>Its important to change all numbers as indicated or none!</b>

In [ ]:
length = 100
tf.random.set_seed(42)
train_set = to_dataset(encoded[:1_000_000], length=length, shuffle=True,
                       seed=42) # change the 1_000_000 to 100_000 to make it faster
valid_set = to_dataset(encoded[1_000_000:1_060_000], length=length) # change the 1_000_000 to 100_000 and the 1_060_000 to 106_000 to make it faster
test_set = to_dataset(encoded[1_060_000:], length=length) # change 1_060_000 to 112_000 to make it faster

![Heading that says "Model"](images/model_heading.png)
`Image source: own illustration`

# 3. Building and Training the Model  
In the following step, the AI model is being built. With the type neural network, there are different layers being added. Understanding the layers in detail is very complicated and involves a lot of mathematics. A few important notes about the following code can be made though. Firstly, the first layer of a neural network is an activation layer that can have multiple types. For this model, it is a softmax layer which can broadly be said to transform the data into probabilities, which is neccessary as we need the words most likely to come after the input words as the output (See: Koech, Kiprono (2020): Softmax Activation Function — How It Actually Works. Medium. Avaliable Online at https://towardsdatascience.com/softmax-activation-function-how-it-actually-works-d292d335bd78 (Accessed 06.12.2023).)  
The code also uses Adam as an `optimizer`. The optimizer is pert of what the model does without human command: the optimizer determines which nodes (the AI's neurons) are more important than others, it changes the `weights`. It independently adapts the neural network to perform better and better each run through. There are different kinds of optimizers but Adam is the staple one (See: Brownlee, Jason(2021): Gentle Introduction to the Adam Optimization Algorithm for Deep Learning. Machine Learning Mastery. Avaliable Online at https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/ (Accessed 06.12.2023).)  
During training, the model goes through `epochs`, which are specified in the code below to be 10. An epoch is the model going through the entire training dataset once. Overall, the model thus goes through the dataset 10 times, optimizing its setup each time to perform better and better.  
You can watch the performance get better after each epoch by observing the `loss` and `accuracy` values change. These values are created in the following code as well. The accuracy describes how acccurate the predictions of the model are. The loss describes how far away from the desired output the model was.  

<b>`Before executing:`</b>   
Executing the following code will take a while. Depending on your processor it can take up to 12 hours. Make sure there are no other programs running in the background, this will make it even slower. It is also important that the computer does not set itself into standby or runs out of battery, as this will set back any progress. Dont restart the code either.  
Grab a nice hot drink and sit back to train the model. 

In [ ]:
tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax") # activation layer is set to be a softmax layer
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    "my_shakespeare_model", monitor="val_accuracy", save_best_only=True)
history = model.fit(train_set, validation_data=valid_set, epochs=10,
                    callbacks=[model_ckpt])

In case you run into errors while training, make sure to read the `error message` carefully. A very good place to look up solutions is <a href="https://stackoverflow.com/"> Stack Overflow </a>. Sometimes, error messages might pop up in red but can be ignored as they refer to potential future problems that we wont run into, since we wont work with the model furhter. 

### Reflecting the Training  
Starting the training, in the first epoch the accuracy is at 0.4. This might be surprisingly low. Over all epochs, it does climb to 0.7 which is significantly better than the beginning but could be even higher. When using the model in the following steps, it is important to keep this in mind as it will reflect in the output. The fact that the accuracy is improving though, is a good sign. This means the model is learning!  
Since this neural network is trained on a very small scale, this could be the reason for the improvable accuracy. Usually, when training a model, training takes place on a way bigger scale with more epochs and more data.  
Nonetheless, the model is good enough to use in the next step!

Before using the model to get some output, it has to be saved.

In [ ]:
shakespeare_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X - 2),  # no <PAD> or <UNK> tokens
    model
])

## Interlude: Peer Discussion  
The time it takes to run a model might be subjet to a peer discussion as it carries a number of implications.  
For one, the time the model takes stands in relationship to the performance of the model in the sense that changing the code to make training faster often negatively impacts accuracy. There is a belance between the two that has to be found in machine learning. This balance needs to be reflected thoroughly and set very transparent to show how the project was amended to comply with the limited resources. <b> How much can we justify reducing training time in cost of accuracy? </b>  
This shows the impact that the avaliable resources (time, money, computing power) have on the model's performance. These resources are often something that research projects in the Arts and Humanities do not have. Moreover, research projects might need financial sponsorship. This puts into question, whether such projects are still idependent research. <b> How independent should AI resarch be? </b>  
With any resarch that is tied to finite resources, the question of accessibility of resarch and knowledge is raised. To prevent AI becoming inaccessible and tied to financial means, AI and machine learning knowledge needs to be spread to different disciplines, like in this notebook.

![Heading that says "Generating"](images/generating_heading.png)
`Image source: own illustration`

# 4. Generating Text
Coming to the final part of the notebook, this is where it gets fun! In the following code, we can actually test out the performance of our very own AI model and see how good it knows Shakespeare.   
<b>`Code explanation:`</b>  
The fist line specifies our input in the model. We want the model to complete the sentence "To be or not to b" and the numbers specify, that we only one the last letter. Expecting an "e", lets see what the model puts out!

In [ ]:
y_proba = shakespeare_model.predict(["To be or not to b"])[0, -1]
y_pred = tf.argmax(y_proba)  # choose the most probable character ID
text_vec_layer.get_vocabulary()[y_pred + 2]

That looks good! The next code it to calculate probabilities and draw 8 samples. Then we define a `function` called next_char that allows us to specify a parametre "temperature" that lets us manipulate the output more. 

In [ ]:
log_probas = tf.math.log([[0.5, 0.4, 0.1]])  # probas = 50%, 40%, and 10%
tf.random.set_seed(42)
tf.random.categorical(log_probas, num_samples=8)  # draw 8 samples

In [ ]:
def next_char(text, temperature=1):
    y_proba = shakespeare_model.predict([text])[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0, 0]
    return text_vec_layer.get_vocabulary()[char_id + 2]

In the next code, we define another function that extend the text to more characters than just the "e" like above. We do still need to include more code that allows reproducibility again. 

In [ ]:
def extend_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [ ]:
tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU

Lets generate text longer than just a letter using the extend_text function. Will our model be able to extend the sentence?

In [ ]:
print(extend_text("To be or not to be", temperature=0.01))

In the next attempt, we increase the temperature to 1. How much will the output change?

In [ ]:
print(extend_text("To be or not to be", temperature=1))

Lets go extreme and see how much the output changes with a temperature of 100.

In [ ]:
print(extend_text("To be or not to be", temperature=100)) # change the temperature value here

![Heading that says "Reflection"](images/reflection_heading.png)
`Image source: own illustration`

# 5. Conclusion
Wrapping things up, there are a few steps in machine lerning that come after the application of the model.  
![Shakespeare saying the last output](images/output_illustration.png)  
`Image source: own illustration`  
### Reflection of Results  
It is important to critically reflect what the odel put out and how this relates to the input.   
When looking at the results, one can definetely see the impact that the temperature has on the output. The step between 1 and 100 is very big and as expected the change in the output is very big. But instead of being much more accurate, the output becomes distorted and nonsensical. Something like <i>"To be or not to bef ,mt'&o3g:adm-$
wh-nse?pws3ert--vgtsdjw!c-yje,znj"</i> is very far from Shakespeare and the model breaks, including non-alphabetic characters. Even with a temperature of 0.1, the model already puts out words that are not real.  
Exploring the temperature further, one could decrease the value and change it to lower than 0.1. In the code above, it is indicated where this change can be made. It could be interesting to see whether this impacts the performance of the model to be better.  
Assuming this model behaves like other neural networks, it might improve with the temperature decreasing but reach a limit of improvement at a certain number. This limit also exists with an increasing temperature, hitting a point where the output includes non-alphabetic characters. Exploring and finding this area of the model working is an important step to evaluating the capabilites of the model.  
Apaprt from evaluating the results based on the characters it includes and how close to the english language it is, we can also evaluate it in comparison to the actual way the Shakespeare quote continues. The first part of Hamlet's famous soliloquy is:  
<br> 
<i>"To be, or not to be: that is the question:
Whether ’tis nobler in the mind to suffer
The slings and arrows of outrageous fortune,
Or to take arms against a sea of troubles,
And by opposing end them?" </i>   
(See: SShakespeare, William, 1564-1616 author. (1954). The tragedy of Hamlet, Prince of Denmark. [London] :The Folio Society.)  <br>  
Coparing this to the output gained from the temperature being 0.1 (<i>"To be or not to be promide, ere we ourry good madam, as a loves the"</i>, one does not find as many similarities regarding the content either.  
In a peer discussion, other means to evaluate the model's performance can be discussed. ASpects like showing the output to an unknoing third party and asking them to assume which author wrote the quote could be a measure. One can assume that the quotes would still be sttributed to Shakespeare, which would mean the model did perform in that sense. 

### Other Use-Cases and Data
After having gone through the steps to build a model of such abilities, the good thing is that one can use it again.  
It would be really interesting to use other authors' data and have the model train with it. This could reveal if the model is jsut coincidentally very good with Shakespeare or performs with other authors as well. It would be interesting to train it with modern language as well.  
Generally, a network like this could be useful in many different disciplines. An example woule be an educational context as it reveals what language is very specific to certain authors. One can see the special characteristics of Shakespeare's writing style, like the cut off words. Having a model that can complete prompts from a selection of different authors might be really useful in a classroom for children as it provides opportunities for questioning why or why not a promt completion sounds like an author or not. It also allows a critical attitude towards AI output being formed in children when they see the nonsensical output, which is important with models like ChatGPT on the rise.  
Changing the data, it is crucial not to forget all the steps we took working with the data and inspecting it. Every time the data changes, a new reflection about its origin and potential bias is important. Additionally, the data needs to be encoded and tokenized again.

### Ethical Concerns
An area of AI research that is often overlooked, but relates very closely to the Arts and Humanities, is AI ethics. Even in this small-scope AI project, there are a few ethical concerns that need to be reflected.  
To begin with, working with data in general always throws `copyright` into question. In the training data, we are using the works of William Shakespeare. In this specific case, his writing has been placed under the creative commons license, meaning they are open to use for all. However, this is not the case for a lot of authors and replacing the input data, it is legally and ethically important to consider copyright. In the special case of deceased authors, one cannot ask them anymore, but it might be the case that authors dont want their writing to be used in a neural network. Similar, when using other peoples' work as a basis, it is ethically important to  consider whether or not the use cases remain true to the spirit of the creator.   
Moreover, the copyright question for the output of the model is important. AI generated works currently stir up the legal community worldwide as the question of whose intellectual property the output is, is heavily debated.  
Answering this question is complicated but what is certain from an ethical perspective is that the output based on someone elses creative work should for now not be used for monetary gain.  
Having grazed the topic of balance previously already, it is always important for AI projects to reflect on the gain versus cost aspect. What does one stand to gain from training a model in contrast to the costs. The costs do not only include resources like time and actual money, but also the electricity it takes to run computers for the training time.  
Unfortunataly, with the climate crisis we are facing right now, it is impossible to not consider the quite high environmental impact of AI. Training a large scope model generates a substantial carbon footprint. An introduction on the environmental impact of AI can be found in <a hre= "https://www.forbes.com/sites/bernardmarr/2023/03/22/green-intelligence-why-data-and-ai-must-become-more-sustainable/">this Forbes article</a>.  
(Marr, Bernard (2023): Green Intelligence: Why Data And AI Must Become More Sustainable. Forbes. Avaliable Online at https://www.forbes.com/sites/bernardmarr/2023/03/22/green-intelligence-why-data-and-ai-must-become-more-sustainable/ (Accessed 06.12.2023).)  
  
In this case, the educational gain of training the model justifies implementing something that has been done before. Especially the small scope of the training process means the environmental impact is not outweighing the gain throught the model's building and training.

### Final Words
The ethical deliberation is concluding this tutorial. Even on this small scope, a generation of text output was possible and the potential of the model can already be seen. You are welcome to start exploring fully on your own, the world of AI is big! Changing this code to see how the performance might be impacted could be a valid starting point. The batch size, training data size and epoch number could be interesting to change and see how the accuracy is affected. Otherwise, a few links below proide starting points into programming your own AI model independently.

## Furhter information:
- <a href="https://stackoverflow.com/">Stack Overflow</a>
- <a href="https://www.tensorflow.org/learn">Introduction to Tensorflow</a>
- <a href="https://www.tensorflow.org/text/tutorials/text_generation">Text generation with Tensorflow</a>
- <a href="https://medium.com/mlearning-ai/build-your-own-ai-powered-content-generator-with-7-lines-of-python-code-a679e7d5a563">Medium tutorial</a>



![Shakespeare saying "good job!"](images/good_job.png)
`Image source: own illustration`